In [4]:
# Gabriel Chang

# funcion para descargar paquetes al ejecutar el scrip, sin necesidad de hacerlo por la terminal
# igualmente hice esto en la última tarea, solo por cuestión de mejorar el ejecución
# en diferentes máquinas

import os
import subprocess
import sys

# en ese arreglo incluyo los paquetes que debo instalar, para facilitar el mantenimiento
# (las necesarias para la asignacion de EDA U3T2)
required_packages = ['pandas', 'matplotlib', 'numpy', 'seaborn']

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [5]:
# hago un for each iterando en el arreglo de paquetes requeridos, y los instalo
# si es que no lo están

for package in required_packages:
    try:
        __import__(package)
        print(f"'{package}' ya está instalado.")
    except ImportError:
        print(f"'{package}' no está instalado. Instalando...")
        install(package)

'pandas' ya está instalado.
'matplotlib' ya está instalado.
'numpy' ya está instalado.
'seaborn' ya está instalado.


In [6]:
# Import de librerias e inicialización del data frame
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sea
import numpy as np

In [10]:
# data set --> Technical Support Dataset
url = 'https://raw.githubusercontent.com/spamella04/InteligenciadeNegocios2024/refs/heads/Gabriel/GCHANG%20-%20U3T2/CO2%20emission%20by%20countries.csv'

try:
    data_frame = pd.read_csv(url, encoding='unicode_escape')

    # Crear una copia del dataset original para modificar
    data_frame_modificado = data_frame.copy()

    print('Archivo CSV cargado al Data Frame exitosamente')

    print('\nIMPRIMIENDO HEAD DEL DATA SET\n :', data_frame_modificado.head())

except Exception as ex:
    print('ERROR AL CARGAR EL DF --> ', ex)

Archivo CSV cargado al Data Frame exitosamente

IMPRIMIENDO HEAD DEL DATA SET
 :        Country Code Calling Code  Year  CO2 emission (Tons)  Population(2022)  \
0  Afghanistan   AF           93  1750                  0.0        41128771.0   
1  Afghanistan   AF           93  1751                  0.0        41128771.0   
2  Afghanistan   AF           93  1752                  0.0        41128771.0   
3  Afghanistan   AF           93  1753                  0.0        41128771.0   
4  Afghanistan   AF           93  1754                  0.0        41128771.0   

       Area % of World Density(km2)  
0  652230.0      0.40%       63/km²  
1  652230.0      0.40%       63/km²  
2  652230.0      0.40%       63/km²  
3  652230.0      0.40%       63/km²  
4  652230.0      0.40%       63/km²  


In [11]:
# Cantidad de registros:
print('CANTIDAD DE REGISTROS (FILAS) EN EL DATA SET --> ', data_frame_modificado.shape[0])

# Cantidad de columnas:
print('\nCANTIDAD DE COLUMNAS (VARIABLES) EN EL DATA SET --> ', data_frame_modificado.shape[1])

# Descripcion de los datos
print(f'\nIMPRRIMIENDO NÚMERO DE FILAS Y TIPOS DE DATOS:\n')
print(data_frame_modificado.info())

# Estadísticas básicas
print(f'\nIMPRMIENDO ESTADÍSTICAS BÁSICAS DEL DATA FRAME:')
print(data_frame_modificado.describe().T)

CANTIDAD DE REGISTROS (FILAS) EN EL DATA SET -->  59620

CANTIDAD DE COLUMNAS (VARIABLES) EN EL DATA SET -->  9

IMPRRIMIENDO NÚMERO DE FILAS Y TIPOS DE DATOS:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59620 entries, 0 to 59619
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              59620 non-null  object 
 1   Code                 57452 non-null  object 
 2   Calling Code         56097 non-null  object 
 3   Year                 59620 non-null  int64  
 4   CO2 emission (Tons)  59620 non-null  float64
 5   Population(2022)     53116 non-null  float64
 6   Area                 55284 non-null  float64
 7   % of World           55284 non-null  object 
 8   Density(km2)         53116 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 4.1+ MB
None

IMPRMIENDO ESTADÍSTICAS BÁSICAS DEL DATA FRAME:
                       count          mean           std      min